In [11]:
import numpy as np
import scipy
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import pylab as plt
%matplotlib inline

#EPSILON=0.5

# Datum
trainX = np.loadtxt('X_train.csv', delimiter=',', skiprows=1)
traint = np.loadtxt('y_train.csv', delimiter=',', skiprows=1)
traint = traint[:,1][:,None]-1

testX = np.loadtxt('X_test.csv', delimiter=',', skiprows=1)

"""def plot_pair(X, t, f1, f2):
    plt.figure()
    pos0 = np.where(t==0)[0]
    pos1 = np.where(t==1)[0]
    plt.plot(X[pos1,f1], X[pos1,f2],'bo')
    plt.plot(X[pos0,f1], X[pos0,f2],'ro')"""

def save_predictions(predictions, filename="class_logit.csv"):
    N = predictions.shape[0]
    
    output = np.ones((N, 2))
    output[:,0] = range(N)
    output[:,1] = predictions.ravel()
    np.savetxt(filename, output, fmt='%d', delimiter=",", header="Id,EpiOrStroma")
    
def to_binary(x):
    if x < 0.5:
        return 0
    return 1

def h(x, w):
    return sigmoid(np.dot(x, w))

def sigmoid(x):
    val = scipy.special.expit(x)
    return val

#a = np.array([[3,3],[-4,-4],[5,5]])
#w2 = np.array([2,2])[:,None]
#print(to_binary(h(a,w2)))

def cv(trainX, traint, folds=20):
    sample_count = trainX.shape[0]
    fold_size = sample_count//folds
    
    average_accuracy = 0
    
    for i in range(folds):
        split_index1 = i*fold_size
        split_index2 = (i+1)*fold_size
        
        cv_testX = trainX[split_index1:split_index2]
        cv_testt = traint[split_index1:split_index2]
        cv_trainX = np.concatenate((trainX[:split_index1], trainX[split_index2:]))
        cv_traint = np.concatenate((traint[:split_index1], traint[split_index2:]))
        
        #
        # Homemade models
        #        
        #w = get_nr_weights(cv_trainX, cv_traint)
        w = get_gd_weights(cv_trainX, cv_traint)
        predictions = np.array(list(map(to_binary, h(cv_testX, w))))[:,None]
        #print(predictions)
        
        #
        # Sklearn models
        #
        #model = get_sk_model(cv_trainX, cv_traint)
        #predictions = model.predict(cv_testX)
        #print(predictions[0:5])
        #print(cv_testt[0:5])
        
        acc = get_accuracy(predictions, cv_testt)
        average_accuracy += acc
        print("Iteration and accuracy:", i, acc)
    
    print("Total accuracy:", average_accuracy/folds)
    
def get_accuracy(predictions, actual):
    N = predictions.shape[0]
    temp = predictions-actual
    misses = np.count_nonzero(temp)
    hits = N-misses
    return hits/N
        
def make_predictions(X, w):
    preds = np.zeros((X.shape[0], 1))
    for i in range(X.shape[0]):
        x = X[i]
        preds[i] = to_binary(h(x, w))
    return preds
        
def filter_by_ttest(trainX, traint, n=10):
    ps = np.zeros(trainX.shape[1])
    
    for i in range(trainX.shape[1]):
        feature = trainX.T[i][:,None]
        label = traint.flatten()[:,None]
        
        stacked = np.hstack((feature, label))
        stacked.sort(axis=0)
        count = (stacked[:,1] == 1).sum()
        group1 = stacked[:count,0]
        group2 = stacked[count:,0]
        
        p = stats.ttest_ind(group1,group2)[1]
        ps[i] = p
        
    features_by_distinction = ps.argsort()[:n]
    #print(features_by_distinction)
    newX = np.zeros((trainX.shape[0], n))
    
    for i in range(n):
        newX[:,i] = trainX[:,features_by_distinction[i]]
    
    return newX, features_by_distinction

def filter_by_variance(trainX, keep_count=10):
    sample_count = trainX.shape[0]
    feature_count = trainX.shape[1]
    variances = np.zeros(feature_count)
    
    # Find variance of each feature
    for i in range(feature_count):
        feature = trainX.T[i]
        variance = np.var(feature)
        variances[i] = variance    
    
    features_by_variance = variances.argsort()[:keep_count]

    newX = np.zeros((sample_count, keep_count))
    for i in range(keep_count):
        newX[:,i] = trainX[:,features_by_variance[i]]

    return newX, features_by_variance

def get_sk_model(trainX, traint):
    clf = LogisticRegression()
    model = clf.fit(trainX, traint.ravel())
    return model

"""def predict_sklearn(model, testX):
    predictions = model.predict(testX)
    return predictions"""

# Newton Raphson
def get_nr_weights(trainx, traint, iterations=20):
    n_weights = trainx.shape[1]
    
    w = np.zeros((n_weights,1))
    dx = np.diag(np.dot(trainx, trainx.T))[:,None]
    
    # allw - for debugging
    #allw = np.zeros((n_weights,iterations))

    for i in range(iterations):
        #allw[:,i] = w.flatten()
        P = 1.0/(1.0 + np.exp(-np.dot(trainx, w)))
        gw = -w + np.sum(trainx*np.tile(traint-P,(1,n_weights)), axis=0)[:,None]
        temp = trainx*np.tile(P*(1-P), (1,n_weights))
        hw = -np.eye(n_weights) - np.dot(temp.T, trainx)
        w = w - np.dot(np.linalg.inv(hw), gw)
    
    # Debug
#     [plt.plot(allw[i,:]) for i in range(100)]
#     plt.xlabel('Iteration')
#     plt.ylabel('w')
    
    return w

# Gradient descent
def get_gd_weights(X, t, iterations=200000, alpha=0.9):
    sample_count = X.shape[0]
    weight_count = X.shape[1]
    w = np.zeros(weight_count)[:,None]
    
    # Plot
    #allw = np.zeros((weight_count,iterations))
    
    for i in range(iterations):
        #allw[:,i] = w.flatten()
        w -= alpha/sample_count * np.dot(X.T, h(X,w)-t)

        # Reduce alpha
        #alpha *= 0.99
        
        # Debug
        if i%10000 == 0: print(w[0:5].T, i)
            
        #print("Updated weights after iteration", iters)

#     [plt.plot(allw[w,:]) for w in range(weight_count)]
#     plt.xlabel('Iteration')
#     plt.ylabel('w')

    return w

# Filter by ttest
#trainX, top_feature_indices = filter_by_ttest(trainX, traint, 25)
trainX, top_feature_indices = filter_by_variance(trainX, 22)

#w = get_nr_weights(trainX, traint)
#w = get_gd_weights(trainX, traint, 300000, 0.9)


#
# Below are scores from Kaggle
#
#w = get_gd_weights(trainX, traint, 300000, 0.9) (22 features by variance) = 0.83054



# Single CV
#cv(trainX, traint)

# Loop CV to find subset of features
# for feature_count in range(1, 112):
#     newX, _ = filter_by_ttest(trainX, traint, feature_count)
#     #newX, _ = filter_by_variance(trainX, feature_count)
#     cv(newX, traint)










def save_predictions(predictions, filename="classification_predictions.csv"):
    N = predictions.shape[0]
    
    output = np.ones((N, 2))
    output[:,0] = range(N)
    output[:,1] = predictions.ravel()
    np.savetxt(filename, output, fmt='%d', delimiter=",", header="Id,EpiOrStroma")
    print("Predictions saved")

# Select only top features if filtered
testX = testX[:,top_feature_indices]

# Uncoment line below to predic w with logistical
#preds = make_predictions(testX, w)

# Uncomment block below to predict w with sklearn
model = get_sk_model(trainX, traint)
preds = model.predict(testX)

# Save
#save_predictions(preds+1)

[[0.00021211 0.04353853 0.09906561 0.01390315 0.03327729]] 0
[[ -0.17198633   2.00744063   4.98585083   1.7600611  -17.08701797]] 10000
[[ -0.3497748    2.89475191   7.18886141   2.58879266 -25.64888126]] 20000
[[ -0.52030017   3.5257375    8.93647398   3.19475496 -30.80830352]] 30000
[[ -0.68601468   3.97018999  10.42922231   3.65845149 -34.25035449]] 40000
[[ -0.84872688   4.26169539  11.72884997   4.02069901 -36.70244043]] 50000
[[ -1.00959282   4.42449327  12.86898011   4.30565439 -38.53139583]] 60000
[[ -1.16929798   4.47847171  13.87480282   4.52989746 -39.94337054]] 70000
[[ -1.32824279   4.44053104  14.76734546   4.70570673 -41.06336011]] 80000
[[ -1.48666387   4.32512027  15.56457641   4.84250333 -41.97152624]] 90000
[[ -1.64470466   4.14456374  16.28183186   4.94764817 -42.72157256]] 100000
[[ -1.802455     3.90932997  16.93211592   5.02695682 -43.35077882]] 110000
[[ -1.95997342   3.62827665  17.52638404   5.0850621  -43.88580805]] 120000
[[ -2.11729965   3.30887573  18.0738

[[ -1.13367894   8.25209904  17.71216906   6.48330243 -41.16452446]] 70000
[[ -1.30182253   8.34192358  18.61826225   6.77411774 -42.55658519]] 80000
[[ -1.47076184   8.33978263  19.41042029   7.00674555 -43.72116733]] 90000
[[ -1.64038394   8.26212384  20.11088964   7.19127298 -44.71192198]] 100000
[[ -1.81058667   8.12210645  20.73714647   7.33572042 -45.56662748]] 110000
[[ -1.98127902   7.93037729  21.3030701    7.44656621 -46.3127037 ]] 120000
[[ -2.1523807    7.69561926  21.81976586   7.52910796 -46.97057059]] 130000
[[ -2.32382126   7.42495638  22.29616849   7.58772161 -47.55577516]] 140000
[[ -2.49553894   7.12426202  22.73949745   7.62605266 -48.08038333]] 150000
[[ -2.66747965   6.79839822  23.15560692   7.64716046 -48.55391683]] 160000
[[ -2.83959593   6.45140419  23.54925831   7.65362891 -48.98399961]] 170000
[[ -3.0118461    6.08664631  23.92433433   7.64765241 -49.3768141 ]] 180000
[[ -3.18419346   5.70693852  24.28400832   7.63110356 -49.73742975]] 190000
Iteration and a

[[ -2.10760619  15.23016338  19.58822248   3.46772158 -56.57366102]] 140000
[[ -2.24993201  15.42371629  19.91003164   3.34007172 -57.49813682]] 150000
[[ -2.39154199  15.58159849  20.19942779   3.20584002 -58.3488265 ]] 160000
[[ -2.53242888  15.70832945  20.46171451   3.06726579 -59.13466623]] 170000
[[ -2.67258824  15.80773044  20.70127872   2.92619378 -59.8630594 ]] 180000
[[ -2.81201868  15.88304776  20.92175654   2.78414013 -60.5402108 ]] 190000
Iteration and accuracy: 10 0.8666666666666667
[[0.00022101 0.04965478 0.11105855 0.01503541 0.03934167]] 0
[[ -0.14953347   3.60305539   6.7866378    2.49961781 -17.69354185]] 10000
[[ -0.31457034   5.18153963   9.81937534   3.82741218 -27.06945828]] 20000
[[ -0.481404     6.26387844  12.0664953    4.78890954 -33.07173755]] 30000
[[ -0.64934361   7.04012214  13.89443254   5.514916   -37.32506947]] 40000
[[ -0.81808768   7.59248041  15.43630285   6.07711458 -40.53846343]] 50000
[[ -0.98756445   7.96973054  16.76140852   6.51737718 -43.0739

[[ -0.16506556   3.61011656   6.19969466   2.17707909 -17.34590263]] 10000
[[ -0.33929597   5.30878666   9.17616871   3.33438242 -26.70862469]] 20000
[[ -0.51161856   6.52163913  11.48475851   4.18953551 -32.72346731]] 30000
[[ -0.68288237   7.41409435  13.3775244    4.84211891 -36.97566803]] 40000
[[ -0.85372405   8.06802555  14.96140866   5.34883054 -40.17134281]] 50000
[[ -1.02458112   8.53599998  16.30279054   5.74431725 -42.67497088]] 60000
[[ -1.19570122   8.85557437  17.44979849   6.05200805 -44.69633539]] 70000
[[ -1.36719985   9.05502248  18.43950694   6.28891314 -46.36612989]] 80000
[[ -1.53911058   9.15627179  19.30122248   6.46797003 -47.77073519]] 90000
[[ -1.71141955   9.17665377  20.05840699   6.59932477 -48.96990358]] 100000
[[ -1.88408766   9.13004945  20.7299794    6.69110145 -50.00648417]] 110000
[[ -2.05706403   9.02768612  21.33126754   6.74990152 -50.91211413]] 120000
[[ -2.23029407   8.87871468  21.87473292   6.78114795 -51.71072086]] 130000
[[ -2.40372398   8.69